# Playing with RSA

In [ ]:
from math import log2, floor, isqrt
from timeit import timeit
from sympy import randprime, nextprime

from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

## What happens if p and q are too close?

In [ ]:
# p = next_prime(2^512)
p = randprime(2**511, 2**512)
p

In [ ]:
q=nextprime(p)
q

In [ ]:
n = p * q
n

In [ ]:
# n has 1024 bit
log2(n)

In [ ]:
# We start from the square root and look for the next prime
# isqrt is the integer part of the square root
t = isqrt(n)
t

In [ ]:
nextprime(t)

In [ ]:
# Compare with q
q

## How long does it take to generate a key?

In [ ]:
def gen():
    rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    
timeit(gen, number = 100)

## What happens if the exponent is too low?

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=3,
    key_size=1024
)
public_key = private_key.public_key()

e = public_key.public_numbers().e
n = public_key.public_numbers().n
print("e = {}, n = {}".format(e,n))

In [ ]:
message = b"OK"

m = int.from_bytes(message,"big")
c = pow(m,e,n)

print("m = {}, c = {}".format(m,c))

In [ ]:
# Can we find the message?
pow(c,1/3)

In [ ]:
# If we use padding the problem is solved
c = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1( algorithm=hashes.SHA256() ),
        algorithm=hashes.SHA256(),
        label=None
    )
)
print("c = {:d}".format(int.from_bytes(c,"big")))

In [ ]:
private_key.decrypt(
    c,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

## Signature

The key generation is the same as encryption, but we want a fresh key

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096
)
public_key = private_key.public_key()

In [ ]:
message = b"A message I want to sign"
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1( hashes.SHA256() ),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

In [ ]:
try:
    public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1( hashes.SHA256() ),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("OK")
except:
    print("KO")

In [ ]:
message = b"Not the real message"
try:
    public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1( hashes.SHA256() ),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("OK")
except:
    print("KO")

# Laboratory

Measure the key generation time and the signature time for key sizes ranging from 512 to 4096 bits. Plot a graph.